In [ ]:
import synapseclient
import synapseutils
from synapseclient import Project, File, Folder
from synapseclient import Schema, Column, Table, Row, RowSet, as_table_columns
import pandas as pd

In [ ]:
syn = synapseclient.Synapse()
syn.login()

### Install `Levenshtein` module, the documentation for which is [here](https://rapidfuzz.github.io/Levenshtein/)

In [ ]:
pip install levenshtein

### Basic idea:

1. Extract `value` column from `table_AD_model`
2. Loop through each cell in `table_AD_backend`:

    a. In cases where the term does not exist in `value`, suggest 3 alternatives based on Levenshtein distance
    
    b. Otherwise, do nothing

### Create DataFrame from AMP-AD Knowledge Portal - All Data `FileView`

In [ ]:
schema_AD_backend = syn.get('syn11346063')
results_AD_backend = syn.tableQuery(f"SELECT * FROM {schema_AD_backend.id}")  
table_AD_backend = pd.read_csv(results_AD_backend.filepath)

table_AD_backend

### Create DataFrame from AD Data Model

In [ ]:
schema_AD_model = syn.get('syn53010627')
results_AD_model = syn.tableQuery(f"SELECT * FROM {schema_AD_model.id}")
table_AD_model = pd.read_csv(results_AD_model.filepath)

table_AD_model

### Extract `value` column to list for later similarity scoring

In [ ]:
ctrl_vals = table_AD_model['value'].astype(str).tolist()
ctrl_vals

### Extract columns whose values are of the format `stringList`

In [ ]:
sl_cols = table_AD_backend.columns.tolist()
sl_cols = ['study', 'dataType', 'assay','tissue',
            'species', 'sex','grant', 'modelSystemName', 
            'cellType', 'group', 'metaboliteType']

### Function to *clean* columns (remove extra characters)

In [ ]:
def clean_cols(df_1):
    for col in sl_cols:          
        df_1[col] = df_1[col].str.replace("]", "")
        df_1[col] = df_1[col].str.replace("[", "")
        df_1[col] = df_1[col].str.replace("\"", "")
    
    return df_1

### Return new DataFrame without brackets and quotation marks

In [ ]:
nu_df = clean_cols(table_AD_backend)
nu_df

### Import `ratio` module from `Levenshtein`

In [ ]:
from Levenshtein import ratio

### For now, let's just try the process on the `assay` column

In [ ]:
idx = 0

nu_df['assay'] = nu_df['assay'].astype(str)
for assay in nu_df['assay'].tolist():
    if assay in ctrl_vals:
        pass
    else:
        val_to_sim = {str(val): ratio(assay, val) for val in ctrl_vals}
        nu_vts = list(dict(sorted(val_to_sim.items(), key=lambda x:x[1])[-3:]))
        print("Do you want to change '{}' to any of 0: '{}', 1: '{}', or 2: '{}'? (Enter 0, 1, 2, or 'no'. 'q' to quit)".
              format(assay, nu_vts[0], nu_vts[1], nu_vts[2]))
        choice = input()
        if choice == "no":
            pass
        elif choice == '0':
            nu_df.loc[idx, 'assay'] = nu_vts[0]
            color = 'blue'
        elif choice == '1':
            nu_df.loc[idx, 'assay'] = nu_vts[1]
        elif choice == '2':
            nu_df.loc[idx, 'assay'] = nu_vts[2]
        elif choice == 'q':
            break
        print(nu_df.iloc[idx])
    idx += 1

### `Ctrl` + `F` within `nu_df` for the changed values

In [ ]:
nu_df